# Temporal-Difference Learning
* TD method 로 줄여부름
* episodes 경험으로부터 학습
* model-free : MDP transition / rewards 에 대해 모름
* epiosdes가 완벽히 끝나지 않은 상태에서 학습 (bootstraping)
* 왠지 앞으로 이 value function 일 것이라고 추측하는 값이 되도록 update

### MC and TD
* Goal: policy $\pi$ 에 의한 경험으로 부터 $v_\pi$를 학습
* Incremental every-visti Monte-Carlo
  * 실제 return $G_t$ 를 따르도록 value $V(S_t)$를 update
  $$
  V(S_t) \leftarrow V(S_t) + \alpha (G_t - V(S_t))
  $$
* 가장 단순한 temporal-difference learning algorithm: TD(0) (바로 앞 단계만 보는거)
  * 아마도 이런 return 일 것이다 하는 $R_{t+1} + \gamma V(S_{t+1})$ 을 향해서 value $V(S_t)$ 를 update
  $$
  V(S_t) \leftarrow V(S_t) + \alpha (R_{t+1} + \gamma V(S_{t+1}) - V(S_t))
  $$
  * 위에서 $R_{t+1} + \gamma V(S_{t+1})$을 TD target이라 함
  * $\delta_t = (R_{t+1} + \gamma V(S_{t+1}) - V(S_t))$ 를 TD error 라고 함

## MC vs. TD 장단점
* TD는 최종 결과 나오기 전에 학습 가능
  * TD 는 매 스텝마다 online 학습이 가능
  * MC는 episode 가 종료해서 rerturn 을 알아야 함
* TD 는 최종 결과 없이 학습이 되어버림
  * TD는 incomplete sequences 를 학습 할 수 있음
  * MC 는 complete sequences 만 학습 가능
  * (위에서 sequences는 time step 마다 S, A, R, ...)
  * TD는 non-terminating environments 에서 동작
  * MC는 반드시 terminating environments 에서만 동작

## Bias/Variance Trade-Off
* $G_t = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-1} R_T$ 는 $v_\pi (S_t)$의 unbiased estimate 임 (값이 이리저리 변하지 않고 $\pi$에 의해 정해져서 그런듯)
* True TD target $R_{t+1} + \gamma v_\pi (S_{t+1})$ 는 $v_\pi (S_t)$ 의 unbiased estimate 임 ($\pi$에 의한 값이라서?)
* TD target $R_{t+1} + \gamma V(S_{t+1})$ 는 $v_\pi (S_t)$의 biased estimate 임 ($\pi$로 정해지는 게 아니라 bootstraping 으로 일부 값 가져온 것이라서)
* TD target 은 return 에 비해 매우 낮은 variance
  * Return 은 여러 random actions, transitions, rewards에 의존함
  * TD target 은 하나의 random action, transitions, reward 에 의존함

## MC vs. TD 장단점 (2)
* MC 는 높은 variance, zero bias
  * 수렴 잘함
  * (function approximation 이어도)
  * 초기값에 민감하지 않음 (어차피 return도 여러번 잘 관측한 episodes 가져다가 업뎃하니깐)
  * 이해 쉽고 사용 쉽고
* TD는 낮은 variance지만 약간의 bias
  * MC 보다 효과적
  * TD(0) 은 $v_\pi (s)$ 에 수렴
  * (function approximation 일 때 항상 수렴은 아님)
  * initial value 에 민감

## MC vs. TD 장단점 (3)
* TD 는 Markov 특성을 활용
  * Markov environments 일 때 효과적
* MC는 Markov 특성 활용 못함
  * non-Markov environments 일 때 더 효과적

## n-Step Return
* (TD가 n step 만큼 미래의 경험을 discount 해서 value function 만듦)
* $n=1, 2, \infty$에 대하여, n-step returns 에 관해 생각:
\begin{align}
n & = 1 (TD) & G_t^{(1)} & = R_{t+1} + \gamma V(S_{t+1})\\
n & = 2 & G_t^{(2)} & = R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+2})\\
\cdots\\
n & = \infty (MC) & G_t^{(\infty)} & = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-1} R_T
\end{align}
* n-step return 정의
$$
G_t^{(n)} = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n V(S_{t+n})
$$
* n-step temporal-difference learning
$$
V(S_t) \leftarrow V(S_t) + \alpha \big( G_t^{(n)} - V(S_t)\big)
$$

## $\lambda$-return
* n-step $G_t^{(n)}$ 을 $\lambda$ 로 가중치 줘서 묶음
* 가중치는 $(1-\lambda)\lambda^{n-1}$을 사용
$$
G_t^\lambda = (1-\lambda) \sum_{n=1}^\infty \lambda^{n-1} G_t^{(n)}
$$
* Forward-view TD($\lambda$)
$$
V(S_t) \leftarrow V(S_t) + \lambda \big( G_t^\lambda - V(S_t)\big)
$$

## Backward View TD($\lambda$)
* Eligibility traces 라는 heuristics 를 사용
\begin{align}
E_0 (s) & = 0\\
E_t(s) & = \gamma \lambda E_{t-1} (s) + \mathbf{1}(S_t = s)
\end{align}
* 어떤 state s 에 대해서 처음 (타임 스텝 0일 때)에는 0, 방문할 때 마다 1을 더해주고, time step 이 지남에 따라서 $\gamma$랑 $\lambda$ 로 discount 해줌
* 모든 state s에 대해 eligibility 를 저장해둠
* 여러번 방문한 state는 value에 영향 많이 주고, 방문 했던 경험을 잊어준다는 거
* value V(s) 를 업뎃할 때 아래와 같이
\begin{align}
\delta_t & = R_{t+1}+ \gamma V(S_{t+1}) - V(S_t)\\
V(s) & \leftarrow V(s) + \alpha \delta_t E_t(s)
\end{align}

## 등등... 의 내용